# Now that all necessary data have now been extracted, transformed, and dumped to parquet files we can start doing our analyses by reading these parquet files as tables with duckdb (an in process OLAP) and then use these tables to make our transformations and draw insights

In [1]:
import duckdb
import os
from dotenv import load_dotenv
from pathlib import Path

%load_ext autoreload
%autoreload 2

# Create database in order to persist data if one does not already exist. (DO NOT create a `.duckdb` file manually as this will not contain the instructions for duckdb to read and write data from and to)

In [2]:
# conn = duckdb.connect("chronic_disease_analyses.duckdb")

# Load AWS credentials in order for duck db to read parquet files in s3 bucket 

In [3]:
# Build paths inside the project like this: BASE_DIR / 'subdir'.
# use this only in development
env_dir = Path('./').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

True

In [4]:
# load env vars
credentials = {
    "aws_access_key_id": os.environ["AWS_ACCESS_KEY_ID"],
    "aws_secret_access_key": os.environ["AWS_SECRET_ACCESS_KEY"],
    "region_name": os.environ["AWS_REGION_NAME"],
}

In [5]:
duckdb.sql(f"""
    INSTALL httpfs
""")

In [6]:
duckdb.sql(f"""
    LOAD httpfs
""")

In [7]:
duckdb.sql(f"""
    CREATE SECRET (
        TYPE s3,
        KEY_ID '{credentials["aws_access_key_id"]}',
        SECRET '{credentials["aws_secret_access_key"]}',
        REGION '{credentials["region_name"]}',
        ENDPOINT 's3.{credentials["region_name"]}.amazonaws.com'
    );
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

# loading CDI and Population fact tables

In [8]:
cdi_url = "s3://chronic-disease-analyses-bucket/cdi-data-transformed/CDI.parquet/*.parquet"
cdi_url

's3://chronic-disease-analyses-bucket/cdi-data-transformed/CDI.parquet/*.parquet'

In [9]:
query = f"""
    CREATE OR REPLACE TABLE CDI AS
    SELECT *
    FROM read_parquet('{cdi_url}', union_by_name=True, filename=False)
"""
query

"\n    CREATE OR REPLACE TABLE CDI AS\n    SELECT *\n    FROM read_parquet('s3://chronic-disease-analyses-bucket/cdi-data-transformed/CDI.parquet/*.parquet', union_by_name=True, filename=False)\n"

In [10]:
duckdb.sql(query)

In [11]:
duckdb.sql("""
    SELECT * FROM CDI
""")

┌───────────┬─────────┬────────────┬───────────────┬───────────┬────────────────────┬─────────────────────┬────────────┬─────────────────┬─────────────────────┬────────────────────┬──────────────────┐
│ YearStart │ YearEnd │ LocationID │ DataValueUnit │ DataValue │ LowConfidenceLimit │ HighConfidenceLimit │ QuestionID │ DataValueTypeID │      Latitude       │     Longitude      │ StratificationID │
│   int32   │  int32  │  varchar   │    varchar    │  double   │       double       │       double        │  varchar   │     varchar     │       double        │       double       │     varchar      │
├───────────┼─────────┼────────────┼───────────────┼───────────┼────────────────────┼─────────────────────┼────────────┼─────────────────┼─────────────────────┼────────────────────┼──────────────────┤
│      2015 │    2015 │ IL         │ %             │      10.6 │                4.8 │                22.2 │ ALC1_1     │ CRDPREV         │  -88.99771017799969 │  40.48501028300046 │ NH_B_ASIAN    

In [12]:
us_population_file_names = [
    "s3://chronic-disease-analyses-bucket/population-data-transformed/Population_2000_2009.parquet/*.parquet",
    "s3://chronic-disease-analyses-bucket/population-data-transformed/Population_2010_2019.parquet/*.parquet",
    "s3://chronic-disease-analyses-bucket/population-data-transformed/Population_2020_2023.parquet/*.parquet",
]

In [13]:
query = f"""
    CREATE OR REPLACE TABLE Population AS
    SELECT *
    FROM read_parquet({us_population_file_names}, union_by_name=True, filename=False)
"""

In [14]:
duckdb.sql(query)

In [15]:
duckdb.sql("""
    SELECT * FROM Population
""")

┌─────────┬───────┬──────────────────┬───────┬────────────┐
│ StateID │  Age  │ StratificationID │ Year  │ Population │
│ varchar │ float │     varchar      │ int32 │   int64    │
├─────────┼───────┼──────────────────┼───────┼────────────┤
│ AL      │   0.0 │ NH_M_WHITE       │  2000 │      19270 │
│ AL      │   0.0 │ NH_M_WHITE       │  2001 │      19612 │
│ AL      │   0.0 │ NH_M_WHITE       │  2002 │      18731 │
│ AL      │   0.0 │ NH_M_WHITE       │  2003 │      18623 │
│ AL      │   0.0 │ NH_M_WHITE       │  2004 │      18659 │
│ AL      │   0.0 │ NH_M_WHITE       │  2005 │      18816 │
│ AL      │   0.0 │ NH_M_WHITE       │  2006 │      18877 │
│ AL      │   0.0 │ NH_M_WHITE       │  2007 │      19027 │
│ AL      │   0.0 │ NH_M_WHITE       │  2008 │      18937 │
│ AL      │   0.0 │ NH_M_WHITE       │  2009 │      18039 │
│ ·       │    ·  │     ·            │    ·  │          · │
│ ·       │    ·  │     ·            │    ·  │          · │
│ ·       │    ·  │     ·            │  

# Loading CDI dimension tables

#### location table

In [16]:
cdi_location_url = "s3://chronic-disease-analyses-bucket/cdi-data-transformed/Location.parquet/*.parquet"
cdi_location_url

's3://chronic-disease-analyses-bucket/cdi-data-transformed/Location.parquet/*.parquet'

In [17]:
# note that if we only specify a string instead of a list in read_parquet it must be enclosed in a quote or double quotes
query = f"""
    CREATE OR REPLACE TABLE CDILocation AS
    SELECT *
    FROM read_parquet('{cdi_location_url}', union_by_name=True, filename=False)
"""

In [18]:
duckdb.sql(query)

In [19]:
duckdb.sql("""
    SELECT *
    FROM CDILocation
""")

┌────────────┬──────────────────────┐
│ LocationID │     LocationDesc     │
│  varchar   │       varchar        │
├────────────┼──────────────────────┤
│ MT         │ Montana              │
│ NC         │ North Carolina       │
│ MD         │ Maryland             │
│ CO         │ Colorado             │
│ CT         │ Connecticut          │
│ IL         │ Illinois             │
│ WY         │ Wyoming              │
│ NJ         │ New Jersey           │
│ DE         │ Delaware             │
│ DC         │ District of Columbia │
│ ·          │   ·                  │
│ ·          │   ·                  │
│ ·          │   ·                  │
│ KS         │ Kansas               │
│ MO         │ Missouri             │
│ SC         │ South Carolina       │
│ OH         │ Ohio                 │
│ NE         │ Nebraska             │
│ OK         │ Oklahoma             │
│ OR         │ Oregon               │
│ NV         │ Nevada               │
│ ID         │ Idaho                │
│ MA        

#### Stratification table

In [20]:
cdi_stratification_url = "s3://chronic-disease-analyses-bucket/cdi-data-transformed/Stratification.parquet/*.parquet"
cdi_stratification_url

's3://chronic-disease-analyses-bucket/cdi-data-transformed/Stratification.parquet/*.parquet'

In [21]:
query = f"""
    CREATE OR REPLACE TABLE CDIStratification AS
    SELECT *
    FROM read_parquet('{cdi_stratification_url}', union_by_name=True, filename=False)
"""

In [22]:
duckdb.sql(query)

In [23]:
duckdb.sql("""
    SELECT *
    FROM CDIStratification
""")

┌─────────┬─────────────┬──────────────┬──────────────────┐
│   Sex   │  Ethnicity  │    Origin    │ StratificationID │
│ varchar │   varchar   │   varchar    │     varchar      │
├─────────┼─────────────┼──────────────┼──────────────────┤
│ Male    │ All         │ Both         │ B_M_ALL          │
│ Both    │ White       │ Not Hispanic │ NH_B_WHITE       │
│ Both    │ Black       │ Not Hispanic │ NH_B_BLACK       │
│ Both    │ AIAN        │ Not Hispanic │ NH_B_AIAN        │
│ Both    │ Multiracial │ Not Hispanic │ NH_B_MULTI       │
│ Both    │ NHPI        │ Not Hispanic │ NH_B_NHPI        │
│ Both    │ Other       │ Not Hispanic │ NH_B_OTHER       │
│ Female  │ All         │ Both         │ B_F_ALL          │
│ Both    │ All         │ Hispanic     │ H_B_ALL          │
│ Both    │ All         │ Both         │ B_B_ALL          │
│ Both    │ Asian       │ Not Hispanic │ NH_B_ASIAN       │
├─────────┴─────────────┴──────────────┴──────────────────┤
│ 11 rows                               

#### Question table

In [24]:
question_url = "s3://chronic-disease-analyses-bucket/cdi-data-transformed/Question.parquet/*.parquet"
question_url

's3://chronic-disease-analyses-bucket/cdi-data-transformed/Question.parquet/*.parquet'

In [25]:
query = f"""
    CREATE OR REPLACE TABLE Question AS
    SELECT *
    FROM read_parquet('{question_url}', union_by_name=True, filename=False)
"""

In [26]:
duckdb.sql(query)

In [27]:
duckdb.sql("""
    SELECT *
    FROM Question
""")

┌────────────┬─────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────┬──────────┬────────┐
│ QuestionID │ TopicID │                                                   Question                                                   │                      Topic                      │ AgeStart │ AgeEnd │
│  varchar   │ varchar │                                                   varchar                                                    │                     varchar                     │  double  │ double │
├────────────┼─────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────────────────┼──────────┼────────┤
│ ART1_1     │ ART     │ Arthritis among adults aged >= 18 years                                                                      │ Arthritis                               

#### DataValueType table

In [28]:
data_value_type_url = "s3://chronic-disease-analyses-bucket/cdi-data-transformed/DataValueType.parquet/*.parquet"
data_value_type_url

's3://chronic-disease-analyses-bucket/cdi-data-transformed/DataValueType.parquet/*.parquet'

In [29]:
query = f"""
    CREATE OR REPLACE TABLE DataValueType AS
    SELECT *
    FROM read_parquet('{data_value_type_url}', union_by_name=True, filename=False)
"""

In [30]:
duckdb.sql(query)

In [31]:
duckdb.sql("""
    SELECT *
    FROM DataValueType
""")

┌───────────────────┬──────────────────────────────────────────┐
│  DataValueTypeID  │              DataValueType               │
│      varchar      │                 varchar                  │
├───────────────────┼──────────────────────────────────────────┤
│ MEDIAN            │ Median                                   │
│ NMBR              │ Number                                   │
│ AGEADJRATE        │ Age-adjusted Rate                        │
│ PERCAPALC         │ Per capita alcohol consumption           │
│ USD               │ US Dollars                               │
│ AGEADJMEAN        │ Age-adjusted Mean                        │
│ PREV              │ Prevalence                               │
│ MEAN              │ Mean                                     │
│ AVGANNCRDRATE     │ Average Annual Crude Rate                │
│ CRDRATE           │ Crude Rate                               │
│ AGEADJPREV        │ Age-adjusted Prevalence                  │
│ AVGANNAGEADJRATE  │ Ave

# Loading Population dimension tables

#### State table 

In [32]:
population_state_url = "s3://chronic-disease-analyses-bucket/population-data-transformed/State.parquet/*.parquet"
population_state_url

's3://chronic-disease-analyses-bucket/population-data-transformed/State.parquet/*.parquet'

In [33]:
query = f"""
    CREATE OR REPLACE TABLE PopulationState AS
    SELECT *
    FROM read_parquet('{population_state_url}', union_by_name=True, filename=False)
"""

In [34]:
duckdb.sql(query)

In [35]:
duckdb.sql("""
    SELECT *
    FROM PopulationState
""")

┌───────────────┬─────────┐
│     State     │ StateID │
│    varchar    │ varchar │
├───────────────┼─────────┤
│ Mississippi   │ MS      │
│ South Dakota  │ SD      │
│ Utah          │ UT      │
│ Kentucky      │ KY      │
│ California    │ CA      │
│ Nebraska      │ NE      │
│ New Hampshire │ NH      │
│ Delaware      │ DE      │
│ Minnesota     │ MN      │
│ Nevada        │ NV      │
│   ·           │ ·       │
│   ·           │ ·       │
│   ·           │ ·       │
│ Michigan      │ MI      │
│ Illinois      │ IL      │
│ Maryland      │ MD      │
│ Alaska        │ AK      │
│ Connecticut   │ CT      │
│ Vermont       │ VT      │
│ Oklahoma      │ OK      │
│ West Virginia │ WV      │
│ Virginia      │ VA      │
│ Massachusetts │ NULL    │
├───────────────┴─────────┤
│   51 rows (20 shown)    │
└─────────────────────────┘

#### Stratification table

In [36]:
population_stratification_url = "s3://chronic-disease-analyses-bucket/population-data-transformed/Stratification.parquet/*.parquet"
population_stratification_url

's3://chronic-disease-analyses-bucket/population-data-transformed/Stratification.parquet/*.parquet'

In [37]:
query = f"""
    CREATE OR REPLACE TABLE PopulationStratification AS
    SELECT *
    FROM read_parquet('{population_stratification_url}', union_by_name=True, filename=False)
"""

In [38]:
duckdb.sql(query)

In [39]:
duckdb.sql(f"""
    SELECT *
    FROM PopulationStratification
""")

┌─────────┬─────────────┬──────────────┬──────────────────┐
│   Sex   │  Ethnicity  │    Origin    │ StratificationID │
│ varchar │   varchar   │   varchar    │     varchar      │
├─────────┼─────────────┼──────────────┼──────────────────┤
│ Female  │ Black       │ Not Hispanic │ NH_F_BLACK       │
│ Female  │ Asian       │ Not Hispanic │ NH_F_ASIAN       │
│ Female  │ NHPI        │ Hispanic     │ H_F_NHPI         │
│ Male    │ Asian       │ Not Hispanic │ NH_M_ASIAN       │
│ Male    │ White       │ Not Hispanic │ NH_M_WHITE       │
│ Male    │ AIAN        │ Not Hispanic │ NH_M_AIAN        │
│ Male    │ Multiracial │ Hispanic     │ H_M_MULTI        │
│ Male    │ Black       │ Not Hispanic │ NH_M_BLACK       │
│ Male    │ Black       │ Hispanic     │ H_M_BLACK        │
│ Female  │ Asian       │ Hispanic     │ H_F_ASIAN        │
│  ·      │   ·         │    ·         │     ·            │
│  ·      │   ·         │    ·         │     ·            │
│  ·      │   ·         │    ·         │